This is a dataset which contains huffington post articles. I trimmed the dataset eventually to only include the summary and the headline columnns.

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_csv('huffpost_news_data.csv')
df = df.sample(frac=.02, random_state=42)
df = df.dropna(subset=['description', 'headline'])

df['description'] = df['description'].str.lower().replace(r'\n',' ', regex=True)
df['headline'] = df['headline'].str.lower().replace(r'\n',' ', regex=True)

# df.reset_index(drop=True, inplace=True)


print(df.info())


# Splitting the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 107 entries, 1186 to 168
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   headline           107 non-null    object
 1   url                107 non-null    object
 2   slug               107 non-null    object
 3   short_description  79 non-null     object
 4   tags               107 non-null    object
 5   source             107 non-null    object
 6   source_title       107 non-null    object
 7   article_section    107 non-null    object
 8   author             100 non-null    object
 9   primary_image      107 non-null    object
 10  language           107 non-null    object
 11  description        107 non-null    object
 12  raw_description    107 non-null    object
 13  created_at         107 non-null    object
 14  modified_at        107 non-null    object
 15  published_at       107 non-null    object
 16  uniq_id            107 non-null    object

In [12]:
!pip install datasets

In [13]:
!pip install transformers huggingface_hub

In [7]:
# model_name = 'facebook/bart-base'
model_name = 'google-t5/t5-small'

from transformers import BartTokenizer, BartForConditionalGeneration, AutoTokenizer
from datasets import Dataset, DatasetDict
from transformers import DataCollatorForSeq2Seq


# tokenizer = BartTokenizer.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

prefix = "title: "
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["description"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["headline"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(test_df)

columns_to_remove = [col for col in train_dataset.column_names if (col != 'description' and col != 'headline')]
print(train_dataset[0])
print(columns_to_remove)
train_dataset = train_dataset.remove_columns(columns_to_remove)

train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)




You are using a model of type t5 to instantiate a model of type bart. This is not supported for all configurations of models and can yield errors.
Some weights of BartForConditionalGeneration were not initialized from the model checkpoint at google-t5/t5-small and are newly initialized: ['decoder.embed_positions.weight', 'decoder.layernorm_embedding.bias', 'decoder.layernorm_embedding.weight', 'decoder.layers.0.encoder_attn.k_proj.bias', 'decoder.layers.0.encoder_attn.k_proj.weight', 'decoder.layers.0.encoder_attn.out_proj.bias', 'decoder.layers.0.encoder_attn.out_proj.weight', 'decoder.layers.0.encoder_attn.q_proj.bias', 'decoder.layers.0.encoder_attn.q_proj.weight', 'decoder.layers.0.encoder_attn.v_proj.bias', 'decoder.layers.0.encoder_attn.v_proj.weight', 'decoder.layers.0.encoder_attn_layer_norm.bias', 'decoder.layers.0.encoder_attn_layer_norm.weight', 'decoder.layers.0.fc1.bias', 'decoder.layers.0.fc1.weight', 'decoder.layers.0.fc2.bias', 'decoder.layers.0.fc2.weight', 'decoder.la

{'headline': 'child brides join push to raise marriage age', 'url': 'https://www.huffpost.com/entry/child-brides-join-push-to-raise-marriage-age_b_5915c7c5e4b0bd90f8e6a449', 'slug': 'child-brides-join-push-to-raise-marriage-age', 'short_description': None, 'tags': 'U.S. News,Legal Issues,U.S. Government Resources,Sex Crimes,Teens,Demography,social work', 'source': 'www.huffpost.com', 'source_title': 'HuffPost', 'article_section': 'Politics', 'author': 'Stateline', 'primary_image': 'https://img.huffingtonpost.com/asset/5915c7d71500002400db1fba.jpg?ops=1778_1000', 'language': 'en-US', 'description': 'a number of states are considering bills that would raise the age to get married, with sponsors hoping to protect girls from marriages pushed by parents and predators.  by rebecca beitsch most americans think of child marriage as a vestige of a bygone era. and yet in every state, people under 18 are allowed to marry. some states set minimum ages for brides and grooms — sometimes as low as 13

Map:   0%|          | 0/96 [00:00<?, ? examples/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

In [15]:
print(train_dataset[0])

train_dataset = train_dataset.remove_columns(["description", "headline"])

print(train_dataset[0].keys())

{'headline': 'child brides join push to raise marriage age', 'description': 'a number of states are considering bills that would raise the age to get married, with sponsors hoping to protect girls from marriages pushed by parents and predators.  by rebecca beitsch most americans think of child marriage as a vestige of a bygone era. and yet in every state, people under 18 are allowed to marry. some states set minimum ages for brides and grooms — sometimes as low as 13 or 14 — and usually require the permission of a parent, judge, or both before a minor can wed. but laws in about half the states allow children of any age to marry, as long as they receive the proper permission. that may be changing. this year legislators in 10 states have introduced bills to raise the marriage age. proponents say updating marriage laws, which in many states are more than a century old, will help protect children from being pushed into marriages by parents and predators. some lawmakers, disturbed by instan

In [16]:
#!pip install accelerate -U

In [18]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_name, return_tensors="tf")

tf_train_set = model.prepare_tf_dataset(
    train_dataset,
    shuffle=True,
    batch_size=1,
    collate_fn=data_collator,
)


import tensorflow as tf

model.compile(optimizer=optimizer)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [19]:
model.fit(x=tf_train_set, epochs=3)

Epoch 1/3
96/96 [==============================] - 524s 4s/step - loss: 3.4629
Epoch 2/3
96/96 [==============================] - 384s 4s/step - loss: 3.0883
Epoch 3/3
96/96 [==============================] - 387s 4s/step - loss: 2.7512


In [22]:
!pip install rouge-score nltk sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 2.1 MB/s eta 0:00:00


In [8]:
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(test_df)

columns_to_remove = [col for col in train_dataset.column_names if (col != 'description' and col != 'headline')]
eval_dataset = eval_dataset.remove_columns(columns_to_remove)

train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/96 [00:00<?, ? examples/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

In [24]:
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

# Truncate input descriptions to a manageable size if necessary
def truncate_input(description, tokenizer, max_length=1024):
    tokens = tokenizer(description, max_length=max_length, truncation=True, return_tensors="pt")
    return tokenizer.decode(tokens["input_ids"][0])

# Generating summaries with truncated inputs
predictions = []
for item in eval_dataset:
    truncated_description = truncate_input(item["description"], tokenizer)
    summary = summarizer(truncated_description, max_length=150, min_length=5, truncation=True)[0]['summary_text']
    predictions.append(summary)




In [25]:
from datasets import load_metric

bleu_metric = load_metric("sacrebleu")
rouge_metric = load_metric("rouge")

# Prepare actual titles
actual_titles = [item["headline"] for item in eval_dataset]

# Compute BLEU score
bleu_result = bleu_metric.compute(predictions=[pred.split() for pred in predictions], references=[[title.split()] for title in actual_titles])
print("BLEU score:", bleu_result["score"])

# Compute ROUGE score
rouge_result = rouge_metric.compute(predictions=predictions, references=actual_titles)
print("ROUGE scores:", rouge_result)

<ipython-input-25-eee52140c9f9>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric("sacrebleu")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect 

BLEU score: 1.401932491585002
ROUGE scores: {'rouge1': AggregateScore(low=Score(precision=0.023910814495328368, recall=0.36137068739341466, fmeasure=0.04466372254521944), mid=Score(precision=0.03704997295446859, recall=0.48970549652367834, fmeasure=0.06860377427030667), high=Score(precision=0.050708615049006645, recall=0.6056677981109799, fmeasure=0.09284246997907566)), 'rouge2': AggregateScore(low=Score(precision=0.0016679560948154027, recall=0.029545454545454545, fmeasure=0.0031343323737523204), mid=Score(precision=0.006135334534147271, recall=0.09545454545454546, fmeasure=0.011494998781526296), high=Score(precision=0.011288709967673968, recall=0.1659090909090909, fmeasure=0.021024814510269652)), 'rougeL': AggregateScore(low=Score(precision=0.01893062450869686, recall=0.28851100288600295, fmeasure=0.03517563743905959), mid=Score(precision=0.026033451014537558, recall=0.3651629935720845, fmeasure=0.04827189400218815), high=Score(precision=0.033808846134963516, recall=0.450046569592024

I ended up wit ha BLEU score of 1.4 which unless I'm misinterpreting it is extremely low. I tried changing the training methods, but was not able to improve it much. Unfortunatly I was not able to train it with enough data, since I did not have enough ram which I think caused this issue.

It is the same thing with the Rouge score, which is also quite low. I think that this is due to the same issue.


I think that if I was able to train the model on a larger dataset, or use more batches (in a reasonable amount of time), then it would do much better.

# Task 2
One example of a RL problem that we could model as an MDP is a video game. We could use this to create an AI that could play th game. Lets consider the game Super Mario for example:

State space represents each of the video game's states. These include, but are not limited to, position, level, and health. They also include a few binary states, such as jumping, attacking, running, or ducking.

The action space represents all of Mario's actions at any given time. These would include jumping, moving left or right, ducking, and dashing.

The transition model relates the probabilities of moving between any given state when performing each action. For example, the jump action would have a very high probability of changing the state of "jumping" to true. Likewise, the action of going right would likely increase the x position. However, this might be a lower probability since there can sometimes be obstacles in the way.

There are many possible rewards, but one possible option would simply be rewarding getting coins. This would teach the model to try and find coins.



# Task 3
https://github.com/sadighian/recommendation-gym

This dataset uses the movielens dataset to train a deep reinforcement learning model on user recommendations. It uses contiguous user ratings to train a model to predict movies that user will enjoy given movies that they have liked in the past. It uses Deep RL to predict other movies in the dataset.

It uses a Proximal Policy optimization approach in order to refine its suggestions. It takes initial information such as user demographics to inform initial choices, and then takes feedback from the user in order to refine searches.


